# Deep Learning and Neural Networks
Deep Learning is a subset of machine learning algorithms. It is based on neural networks and is capable of learning complex patterns within the data. It's advantage over simple machine learning methods lies in the fact that it can be built by stacking a number of layers of neurons to discover heirarchical patterns of featurese within the data. Thus, it should be able to extract more complex features not otherwise detected by simpler machine learning algorithms

<img src="https://developer.ibm.com/developer/default/articles/an-introduction-to-deep-learning/images/AI-ML-DL.png" width=240 height=240 />

Image source: [IBM Developer](https://developer.ibm.com/articles/an-introduction-to-deep-learning/)

## Neural Networks
The name "Neural Networks" comes from the structure of neurons in our brains, where multiple neurons connect to each other in layers to perform their respective functions. Although the similarity is vague, the name still catches on. To underestand this concept, let's start with explaining what is a neuron.

#### The Neuron
The basic building block of neural network is the neuron (sometimes called perceptron). It can be thought of as simple functions that take inputs and convert them to outputs. This conversion function is known as the activation function. In each neuron, the following steps take place:

* Take the from each variable in the input layer and multiply it by a factor (weight)
* Sum the product of inputs and weights and add a bias term (b)
* Apply a function (activation function) to the final sum

For the sake of understanding how a neuron works, let's assume we have a data for predicting the metastasis of a primary hepatic tumor. We collected four variables to be used for our task. These are the alpha fetoprotein (AFP), number of risk genes, alanine aminotransferase (ALT) and albumin level. The outcome for each patient is determined by an oncologist and is given in the column "Met.". The figure below shows how to use the input variables (coded $X_1$ to $X_4$) to compute the predicted output using one neuron. Each variable is multiplied by a particular weight ($W_1$ to $W_4$), and the product of these multiplications are summed. A bias term ($b$) is added at the end. Then, an activation function is used to convert the result of activation function to outpus (here, I used sigmoid function, but any other activation function should work).

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_neuron.png" width=800 height=256 />

#### Activation Functions
There is a number of activation functions that are commonly used. These include the sigmoid function and rectified linear unit (ReLU) function. Other activation functions include the the sigmoid function, tanh function, leaky ReLU function, step function and others. For further information, please refer to [THIS WEBSITE](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6).

Sigmoid function is an S-shaped function that gives results between 0 and 1, and thus it is most appropriate when the outcome is binary. The ReLU function is a function that gives 0 for $x < 0$, and returns x when $x > 0$


<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_activations.png" width=400 height=140 />

#### Building Neural Networks
The most basic form of neural networks consists of three layers: the input layer, a hidden layer and an output layer. In the hidden layer, neurons are arranged so that they receive input from the input layer and apply their activation functions on the input. The results are then passed to the output layer to give the final output.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_NN.png" width=800 height=266 />

#### Finding the Best Parameters
The final goal is to find the best parameters that make sure that the output of your network ($P(y)$ or $\hat{y}$) is as close as possible to the actual outcome ($y$). So, we first need a method to compute how far off we are in our prediction, and then we try to minimize this difference. This function is known as the loss function (also called cost function), and there are many loss functions out there. The simplest of which is the regression loss function, and is given by $L(\hat{y}, y) = y-\hat{y}$. Mean squared error (MSE) is another loss function that takes the square root of difference and finds the average of that. It is better in that it works regardless of the sign of difference (because of the squaring) and because it takes the error of all cases into account (by averaging). It is given by:

$$MSE(y,\hat{y}) = \frac{\sum_{i=0}^{n}(y-\hat{y})^2} {n}$$

Loss functions are used to validate the model and modify the weights and bias accordingly. The way it does that is through gradient descent, which basically updates the weights and bias so that the final result is closer to the actual outcome. This process is repeated a number of times until the loss function decreases and the best weights and bias are obtained.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_loss.png" width=400 height=210 />

---
### About the Dataset Used
This is a heart disease database, collected from pateints referred to the angiography clinic in Cleveland, US between 1981 and 1984. The original dataset consists of 76 variables. But most studies that used this dataset used a subset of 14, which we will use today. For futher information, please refer to the [this paper](https://pubmed.ncbi.nlm.nih.gov/2756873/)


---
# Neural Networks using Python
### Import data
Import data from ucimlrepo and take a look at the variables

In [ ]:
from utils import *
from ucimlrepo import fetch_ucirepo
from sklearn.neural_network import MLPClassifier

In [ ]:
# fetch dataset 
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets
y = pd.DataFrame(np.where(y>0,1,0),columns = ['outcome'])

all_data = pd.concat([X,y],axis = 1)

vars = heart_disease.variables
vars.iloc[2,4] = 'chest pain type (0:typical angina, 1:atypical angina, 2:non-anginal pain, 4:asymptomatic)'
vars.iloc[6,4] = 'resting ECG (0:normal, 1: ST-wae abnormality, 2: left ventricular hypertrophy)'
vars.iloc[10,4] = 'peak excercise ST-segment slope (0:upsloping, 1: flat, 2:downsloping)'
vars.iloc[12,4] = '3 = normal; 6 = fixed defect; 7 = reversable defect'
vars.iloc[13,2] = 'Categorical'
vars.iloc[13,0] = 'outcome'

vars.style.set_properties(subset=['description'], **{'width-min': '500px'})



### Have a look at categorical and and integer data 

In [ ]:
cats = vars.loc[np.array(vars == 'Categorical'),'name'].tolist()
ints = vars.loc[np.array(vars == 'Integer'),'name'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 5))
fig.suptitle("Numerical/Ordinal variables")
for i in range(len(ints)):
    row = i//3
    col = i-3*row
    col_name = ints[i]
    sns.kdeplot(ax = axes[row][col],data=all_data, x=col_name, fill = True,bw_adjust = 0.75)# try adding /hue='outcome'/
plt.subplots_adjust(wspace=0.3, hspace=0.3)


In [ ]:
#all_data[cats].hist(bins = 10, figsize = (15,10), layout = (4,4), grid = False)

fig, axes = plt.subplots(2, 4, figsize=(15, 5))
fig.suptitle("Categorical variables")
for i in range(len(cats)):
    row = i//4
    col = i-4*row
    col_name = cats[i]
    sns.histplot(ax = axes[row][col],data=all_data, x=col_name, bins = 15)# try adding /hue='outcome'/
plt.subplots_adjust(wspace=0.3, hspace=0.3)


### Convert categorical variables into dummy variables
This is done using the code pd.get_dummies() which converts each categorical variable into a number of variables each of them representing one of the categories as Yes/No

In [ ]:
all_data_dum=pd.get_dummies(all_data,columns=['sex','cp','fbs','restecg','exang','slope','ca','thal'])
all_data_dum.iloc[:,6::] = all_data_dum.iloc[:,6::].astype(int) # convert them to integers of 0/1
all_data_dum


### 
### 3. Create training and test datasets

The following generally applies for all machine learning algorithms:

The first step to do is to <b>split your data into training and testing datasets</b>. The training dataset will be used to train the model and evaluate parameters, while the testing dataset will only be used at the end to test how well did the model learn.

Imputing the data is not necessary for this dataset. However, if your dataset contains missing values, it might be worth imputing so that you do not miss any data point.

After that, the data will be normalized (mean = 0 and sd = 1) so that the model can coverge faster (i.e. reach the best solution faster).

In [ ]:
# Choose features
X = all_data_dum.loc[:,[item for item in all_data_dum.columns if item !="outcome"]]
y = all_data_dum.loc[:,'outcome']

# Divide into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train.copy())
X_test = imp.transform(X_test.copy())

print(f"Size of training dataset = {X_train.shape} and testing dataset = {X_test.shape}")

# scale both X_train and X_test for faster convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Uncomment to further divide the training into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=5)
# X_test = imp.transform(X_test.copy())
# X_val = scaler.transform(X_val)

plot_train_test(X_train,X_test)
#plot_train_test(X_train,X_test, X_val=X_val) # uncomment to show validation number

### Compute MLP Model and Fit it to the Training Set
Create the model using the stochastic gradient descent, which is a method to find best parameters (See last section in the introduction to remind your self; there is no need to understand what is the math behind SGD solver for now).

In [ ]:
"""
# Uncomment this part and comment the part below to show the different validation curves (if you did so in the cell above)
solvers = ['adam', 'sgd']
activations = ['logistic', 'relu']

# Create a list of models. Append the four possible combinations of parameters
models = []
labels = []
val_accuracies = []
for solver in solvers:
    for activation in activations:
        model = MLPClassifier(hidden_layer_sizes=(128,128),max_iter=2000, activation = activation, solver=solver, random_state=0)
        model.fit(X_train,y_train)
        models.append(model)

        # Append accuracy and label (for plotting)
        val_accuracies.append(np.round(model.score(X_val, y_val),3))
        labels.append([f'solver = {solver}, activation = {activation}'])
"""
model = MLPClassifier(hidden_layer_sizes=(128,128),max_iter=2000, activation = 'relu', solver='adam', n_iter_no_change=25,tol = 0.00001, random_state=0,early_stopping=True)# Experiment with <early_stopping = False>
model.fit(X_train,y_train)
accuracy = np.round(model.score(X_test, y_test),3)




### Plot the Results

In [ ]:
"""
# Uncomment this part and comment the part below to show the different validation curves (if you did so in the cells above)
fig, axes = plt.subplots(2,2, figsize=(12, 7))
for ax, label, accuracy in zip(axes.ravel(), labels, val_accuracies):
    ax.set_title(label[0], font='Cambria', size=18)
    ax.plot(model.loss_curve_)
    ax.text(7,0.65, f'Test Accuracy = {accuracy}', font='Cambria', size=14, va='center_baseline')

"""
fig, ax = plt.subplots(figsize=(6, 5))
ax.set_title("Loss Curve", size=18)
ax.plot(model.loss_curve_)
ax.text(0.5,0.7, f'Test Accuracy = {accuracy}', size=14, va='center_baseline')



### Show Accuracy Results

In [ ]:
best_model = model
best_model.probability = True
best_model.fit(X_train,y_train)

y_probas =best_model.predict_proba(X_test)[::,1]

_, ax = plt.subplots(figsize = (6.5,5))
RocCurveDisplay.from_predictions(y_test, y_probas,name="Malignant vs Benign", color="darkred", plot_chance_level=True,ax=ax)
plt.xlabel("False Positive Rate", fontsize = 16)
plt.ylabel("True Positive Rate", fontsize = 16)
ax.set_xticks(labels=['0%', '20%', '40%', '60%', '80%', '100%'], ticks=np.arange(0,1.1,0.2), fontsize = 12)
ax.set_yticks(labels=['0%', '20%', '40%', '60%', '80%', '100%'], ticks=np.arange(0,1.1,0.2), fontsize = 12)
plt.title("Reciever Operating Characterstic for Predicting\nCancer Malignancy", fontsize = 18,weight = 'bold')
plt.legend()

y_pred = model.predict(X_test) # This gives probabilities of y
y_pred_cat = np.where(y_pred>0.5,1,0) # This converts probabilities to either 0 or 1
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_cat).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
pd.DataFrame(np.round([[sens, spec, accuracy]],3), columns=['Sensitivity', 'Specificity', 'Accuracy'])

# Let's try SVM

In [98]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics

param_grid = [
  {'C': np.logspace(-2,6, 10), 'kernel': ['linear','poly']},
  {'C': np.logspace(-2,6, 10), 'gamma': np.logspace(-6, 3, 10), 'kernel': ['rbf']},
 ]

grid_model = GridSearchCV(SVC(), param_grid, scoring="accuracy", cv=StratifiedKFold(n_splits=5))

print ("\nSearching for the model with best parameters ...\n")

# fit the model to the training data set
grid_model.fit(X_train, y_train)

print('Best score achieved during training = ',np.round(grid_model.best_score_,2))

print('\nThis is the model with best parameters found')
grid_model.best_estimator_

In [ ]:
y_pred = grid_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
pd.DataFrame([[sens, spec, accuracy]], columns=['Sensitivity', 'Specificity', 'Accuracy'])

---
### Further reading:
[IBM Developer website](https://developer.ibm.com/articles/an-introduction-to-deep-learning/)

[Activation Functions in Neural Networks](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6)

[Introduction to Deep Learning](https://www.geeksforgeeks.org/introduction-deep-learning/)

[Scikit-Learn Documentation of MLP Function](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)
